In [ ]:
!pip install requests beautifulsoup4

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.model_selection import train_test_split
import re
import json
import random

In [ ]:
bbc_rss_urls = [
    "https://feeds.bbci.co.uk/news/rss.xml",
    "https://feeds.bbci.co.uk/news/world/rss.xml",
    "https://feeds.bbci.co.uk/news/uk/rss.xml",
    "https://feeds.bbci.co.uk/news/business/rss.xml",
    "https://feeds.bbci.co.uk/news/technology/rss.xml",
    "https://feeds.bbci.co.uk/news/science_and_environment/rss.xml",
    "https://feeds.bbci.co.uk/news/health/rss.xml",
    "https://feeds.bbci.co.uk/news/education/rss.xml",
    "https://feeds.bbci.co.uk/news/entertainment_and_arts/rss.xml",
    "https://feeds.bbci.co.uk/news/politics/rss.xml"
]

def get_bbc_links(rss_url):
    response = requests.get(rss_url)
    soup = BeautifulSoup(response.content, features="xml")
    return [item.link.text for item in soup.find_all("item")]

def get_bbc_article_text(url):
    try:
        r = requests.get(url)
        soup = BeautifulSoup(r.content, 'html.parser')
        article = soup.find('article')
        if article:
            paragraphs = article.find_all('p')
            return ' '.join(p.text for p in paragraphs)
        return None
    except Exception as e:
        print(f"Ошибка {url}: {e}")
        return None

bbc_articles = []
seen_urls = set()

for rss_url in bbc_rss_urls:
    links = get_bbc_links(rss_url)
    for link in links:
        if link in seen_urls or len(bbc_articles) >= 250:
            continue
        seen_urls.add(link)
        text = get_bbc_article_text(link)
        if text and len(text) > 300:
            bbc_articles.append(text)
            print(f"BBC: {len(bbc_articles)} статей собрано.")
        time.sleep(1)

# Сохраняем
with open("bbc_articles.txt", "w", encoding="utf-8") as f:
    for article in bbc_articles:
        f.write(article + "\n" + "="*80 + "\n")

BBC: 1 статей собрано.
BBC: 2 статей собрано.
BBC: 3 статей собрано.
BBC: 4 статей собрано.
BBC: 5 статей собрано.
BBC: 6 статей собрано.
BBC: 7 статей собрано.
BBC: 8 статей собрано.
BBC: 9 статей собрано.
BBC: 10 статей собрано.
BBC: 11 статей собрано.
BBC: 12 статей собрано.
BBC: 13 статей собрано.
BBC: 14 статей собрано.
BBC: 15 статей собрано.
BBC: 16 статей собрано.
BBC: 17 статей собрано.
BBC: 18 статей собрано.
BBC: 19 статей собрано.
BBC: 20 статей собрано.
BBC: 21 статей собрано.
BBC: 22 статей собрано.
BBC: 23 статей собрано.
BBC: 24 статей собрано.
BBC: 25 статей собрано.
BBC: 26 статей собрано.
BBC: 27 статей собрано.
BBC: 28 статей собрано.
BBC: 29 статей собрано.
BBC: 30 статей собрано.
BBC: 31 статей собрано.
BBC: 32 статей собрано.
BBC: 33 статей собрано.
BBC: 34 статей собрано.
BBC: 35 статей собрано.
BBC: 36 статей собрано.
BBC: 37 статей собрано.
BBC: 38 статей собрано.
BBC: 39 статей собрано.
BBC: 40 статей собрано.
BBC: 41 статей собрано.
BBC: 42 статей собрано.
B

In [ ]:
API_KEY = "-----"
guardian_articles = []
page = 1

while len(guardian_articles) < 250:
    url = (
        f"https://content.guardianapis.com/search"
        f"?api-key={API_KEY}&page-size=50&page={page}&show-fields=bodyText"
    )
    r = requests.get(url)
    data = r.json()

    for result in data['response']['results']:
        text = result['fields'].get('bodyText', '')
        if text and len(text) > 300:
            guardian_articles.append(text)
            print(f"Guardian: {len(guardian_articles)} статей собрано.")
            if len(guardian_articles) >= 250:
                break
    page += 1
    time.sleep(1)

# Сохраняем
with open("guardian_articles.txt", "w", encoding="utf-8") as f:
    for article in guardian_articles:
        f.write(article + "\n" + "="*80 + "\n")

Guardian: 1 статей собрано.
Guardian: 2 статей собрано.
Guardian: 3 статей собрано.
Guardian: 4 статей собрано.
Guardian: 5 статей собрано.
Guardian: 6 статей собрано.
Guardian: 7 статей собрано.
Guardian: 8 статей собрано.
Guardian: 9 статей собрано.
Guardian: 10 статей собрано.
Guardian: 11 статей собрано.
Guardian: 12 статей собрано.
Guardian: 13 статей собрано.
Guardian: 14 статей собрано.
Guardian: 15 статей собрано.
Guardian: 16 статей собрано.
Guardian: 17 статей собрано.
Guardian: 18 статей собрано.
Guardian: 19 статей собрано.
Guardian: 20 статей собрано.
Guardian: 21 статей собрано.
Guardian: 22 статей собрано.
Guardian: 23 статей собрано.
Guardian: 24 статей собрано.
Guardian: 25 статей собрано.
Guardian: 26 статей собрано.
Guardian: 27 статей собрано.
Guardian: 28 статей собрано.
Guardian: 29 статей собрано.
Guardian: 30 статей собрано.
Guardian: 31 статей собрано.
Guardian: 32 статей собрано.
Guardian: 33 статей собрано.
Guardian: 34 статей собрано.
Guardian: 35 статей соб

In [2]:
# Загрузка текста
with open("news_corpus.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

# Разбиваем по разделителю строк
raw_articles = raw_text.split("================================================================================")

# Убираем теги [BBC], [Guardian], пустые строки и пробелы
cleaned_articles = []
for article in raw_articles:
    # Удаляем лишние теги
    article = article.replace("[BBC]", "").replace("[Guardian]", "").strip()
    if article:
        cleaned_articles.append(article)

# Проверим результат
print(f"Количество статей: {len(cleaned_articles)}")
print("Пример статьи:", cleaned_articles[0][:500])  # первые 500 символов первой статьи

Количество статей: 500
Пример статьи: This week's G7 summit in Canada will be dominated by war - only not one of those that the world leaders had expected. High on the agenda had been Russia's war against Ukraine and Donald Trump's tariff war against America's trading partners. Instead the three-day gathering in the Rocky Mountains of Alberta will inevitably be focused on war in the Middle East. Israel's decision to attack Iran will force the Group of Seven western powers to spend less time on other issues and instead discuss ways o


In [3]:
sentences = []
for article in cleaned_articles:
    # Разбиваем по точкам, удаляем лишние пробелы
    sents = re.split(r'[.!?]', article)
    sents = [s.strip().lower() for s in sents if len(s.strip()) > 10]  # оставим фразы длиной >10 символов
    sentences.extend(sents)

In [4]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)

X = []
y = []

for seq in sequences:
    for i in range(1, len(seq)):
        X.append(seq[:i])
        y.append(seq[i])

# Дополняем входы нулями до одинаковой длины
X = pad_sequences(X)
y = tf.keras.utils.to_categorical(y, num_classes=len(tokenizer.word_index) + 1)


In [5]:
from tensorflow.keras.utils import Sequence
import numpy as np

class DataGenerator(Sequence):
    def __init__(self, X, y, batch_size, vocab_size):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.vocab_size = vocab_size

    def __len__(self):
        return int(np.ceil(len(self.X) / self.batch_size))

    def __getitem__(self, idx):
        batch_x = self.X[idx*self.batch_size:(idx+1)*self.batch_size]
        batch_y = self.y[idx*self.batch_size:(idx+1)*self.batch_size]

        # Если y one-hot encoded, убедитесь, что он имеет правильную форму
        if len(batch_y.shape) == 1:
            batch_y = tf.keras.utils.to_categorical(batch_y, num_classes=self.vocab_size)

        return batch_x, batch_y

In [ ]:
# Использование
batch_size = 64  # Подберите оптимальный размер батча
train_generator = DataGenerator(X, y, batch_size, len(tokenizer.word_index)+1)

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100, input_length=X.shape[1]))
model.add(LSTM(150))
model.add(Dense(len(tokenizer.word_index)+1, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_generator, epochs=20, verbose=1)

Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 860/8147 ━━━━━━━━━━━━━━━━━━━━ 1:34:41 780ms/step - accuracy: 0.0520 - loss: 8.1890